In [27]:
import mechanicalsoup
import json
import hashlib
from bs4 import BeautifulSoup
import sys

BASE_URL = 'https://www.bbc.co.uk'

SCRAPING_SUFFIX = '/iplayer/a-z/'

browser = mechanicalsoup.StatefulBrowser()



In [88]:
# Dictionary to hold the data

program_data = {}

browser.open(BASE_URL + SCRAPING_SUFFIX)

selection = browser.get_current_page().find_all(
    'li', attrs={"class": "grid__item"})

navigation = browser.get_current_page().find('ul', attrs={'class': 'scrollable-nav__track'})

navigation_list = navigation.find_all('li')

navigation_list = [x.a['href'] for x in navigation_list if x.a is not None ]

print(navigation_list)

['/iplayer/a-z/a', '/iplayer/a-z/b', '/iplayer/a-z/c', '/iplayer/a-z/d', '/iplayer/a-z/e', '/iplayer/a-z/f', '/iplayer/a-z/g', '/iplayer/a-z/h', '/iplayer/a-z/i', '/iplayer/a-z/j', '/iplayer/a-z/k', '/iplayer/a-z/l', '/iplayer/a-z/m', '/iplayer/a-z/n', '/iplayer/a-z/o', '/iplayer/a-z/p', '/iplayer/a-z/q', '/iplayer/a-z/r', '/iplayer/a-z/s', '/iplayer/a-z/t', '/iplayer/a-z/u', '/iplayer/a-z/v', '/iplayer/a-z/w', '/iplayer/a-z/y', '/iplayer/a-z/z', '/iplayer/a-z/0-9']


From the initial wep page with the list of programs collect the title and synopsis. Can also grab the number of episodes. Collect link for further extraction

This section yields:
- Title
- Synopsis
- Episode Link



In [90]:
for item in selection:
    title = item.find('p', attrs={"class": "list-content-item__title"})
    synopsis = item.find('p', attrs={'class': 'list-content-item__synopsis'})
    num_episodes_available = item.find(
        'div', attrs={'class': 'list-content-item__sublabels'})
    link = item.find('a', href=True)['href']
    print('link: ',link)
    print('title: ', title.get_text())
    print('synopsis: ', synopsis.get_text())
    
    
    '''
    Follow link for each program item
    '''
    
    browser.open(BASE_URL + link)
    page = browser.get_current_page()

    episode_page_url = page.find('a', attrs={'class': 'lnk'}, text='Credits')
    if episode_page_url is None:
        episode_page_url = browser.get_current_page().find(
            'a', attrs={'class': 'lnk'}, text='Programme website')
    print('latest_episode_url: ',episode_page_url['href'])
    
    
    #navigate to episode page
    browser.open(BASE_URL + episode_page_url['href'])
    episode_page = browser.get_current_page()
    credits = episode_page.find('table', attrs={'class': 'table'})
    
    credits_dict = {}
    if credits is not None:
        for row in credits.find_all('tr'):
            person = row.find_all('span')
            if len(person) > 1:
                json_credits = [x.get_text() for x in person]
                credits_dict[json_credits[0]] = json_credits[1]
        print(credits_dict)

        
    
    similar = episode_page.find(
        'div', attrs={'class': 'footer__similar b-g-p component'})

    if similar is not None:
        sim = similar.find_all('div')
        genre_format = []

        for i in sim:
            genre_format.append(
                [[x.get_text(), x['href']] for x in i.find_all('a')])

        genre_dict={}
        prog_frmt_dict = {}
        genre = []
        prog_frmat = []

        for i in range(len(genre_format)):
            for j in range(len(genre_format[i])):
                if i == 0:
                    genre.append(genre_format[i][j])
                else:
                    prog_frmat.append(genre_format[i][j])

        print('genre: ', genre)
        print('format: ',prog_frmat)


    left_to_watch = episode_page.find(
        'div', attrs={'class': 'grid 1/3@bpw 1/4@bpe'})

    if left_to_watch is not None:
        left_to_watch_items = left_to_watch.find_all(
            'p', attrs={'class': 'episode-panel__meta'})

        if left_to_watch.find(
                'div', attrs={'class': "episode-panel__meta"}) is None:
            if left_to_watch_items[0].span is None:
                days_left = left_to_watch_items[0].get_text()
            else:
                days_left = left_to_watch_items[0].span.get_text()
            duration = left_to_watch_items[1].get_text()

        print('Left to watch: ', days_left,'duration: ' ,duration)
    
    
    
    long_synopsis = episode_page.find(
        'div', attrs={'class': 'synopsis-toggle__long'})

    if long_synopsis is not None:
        long_synopsis_paragraphs = [
            x.get_text() for x in long_synopsis.find_all('p')
        ]

        print('long_synopsis: ',long_synopsis_paragraphs)

    # Broadcast information:

    main_broadcast = episode_page.find(
        'div',
        attrs={
            'class':
            'grid 1/3@bpw2 1/3@bpe map__column map__column--2 map__column--last'
        })

    if main_broadcast is not None:
        date_last_aired = main_broadcast.find(
            'span',
            attrs={'class': 'broadcast-event__date text-base timezone--date'})
        time_last_aired = main_broadcast.find(
            'span', attrs={'class': 'timezone--time'})
        channel = main_broadcast.find(
            'div',
            attrs={
                'class':
                'programme__service box-link__elevated micro text--subtle'
            })
        if channel is not None:
            channel_text = channel.find('a').get_text()
        if channel is not None:
            channel_link = channel.find('a')['href']
        if date_last_aired is not None:
            date_last_aired = date_last_aired.get_text()
        if time_last_aired is not None:
            time_last_aired = time_last_aired.get_text()

        print('date: ', date_last_aired, 'Time: ', time_last_aired, 'Channel: ',
              channel_text, channel_link)
        
    # you may also like
    # navigate to : current page + /recommendations
    # scrape the same way as getting other episode data
    
    current_url = browser.get_url()
    
    browser.open(current_url + '/recommendations')
    
    recommend_page = browser.get_current_page()
    
    if recommend_page is not None:
    
        page_items = recommend_page.find('ol', attrs={'class': 'highlight-box-wrapper'})
        
        if page_items is not None: 
            list_items = page_items.find_all('li')
            for item in list_items:
                item_info = item.find('div', attrs={'class': 'programme__body'})
                link_1 = item_info.h4.a['href']
                link_2 = item_info.h4.a['resource']
                title = item_info.h4.a.get_text()
                synop = item_info.p.get_text()
                
                print('recommendations:', link_1, link_2, 'title: ',title, synop)
    
    
    episodes_link = episode_page.find(
        'a', attrs={
            'class': 'br-nav__link',
            'data-linktrack': 'nav_episodes'
        })
    
    if episodes_link is not None:
        episodes_link = episodes_link['href']

        print(episodes_link)
    
        browser.open(BASE_URL + episodes_link)
        
        
        episodes_page = browser.get_current_page()
        episodes_available_list = episodes_page.find(
            'div', attrs={'class': 'br-box-page programmes-page'})
        episodes_container_list = episodes_available_list.find_all(
            'div',
            attrs={
                'class':
                'programme programme--tv programme--episode block-link highlight-box--list br-keyline br-blocklink-page br-page-linkhover-onbg015--hover'
            })

        available_episodes = episodes_page.find('span', attrs={'class': 'hidden grid-visible@bpb2 grid-visible@bpw'})
        print('available episodes: ', available_episodes.get_text())
        
        for item in episodes_container_list:
            # link
            item_headder = item.find(
                'div', attrs={'class': 'cta cta__overlay'})
            item_link = item_headder.a['href']
            # time left
            item_time_left = item_headder.a['title']
            # title
            item_body = item.find('div', attrs={'class': 'programme__body'})
        
            print(item_link, item_time_left)
        
            if item_body is not None:
                # num of episodes
                try: 
                    episode_oneline_synopsis = item_body.p.get_text()
                    print('episode_oneline_synopsis: ' ,episode_oneline_synopsis)
                except:
                    pass
                try:
                    episode_no = item_body.p.abbr['title']
                    print('episode_no: ', episode_no)
                except:
                    pass
                try:
                    episode_title = item_body.find('span', attrs={'class': 'programme__title gamma'}).get_text()
                    print(episode_title)
                except:
                    pass
        
        
        episode_pagination = episode_page.find('ol', attrs={'class': 'nav nav--banner pagination delta'})
        
        print(episode_pagination)
        
        
        
        
    print('\n')

('link: ', '/iplayer/episode/p02f7cfh/abadas-melin-wynt')
('title: ', u'Abadas')
('synopsis: ', u'Rhaglen animeiddio i blant ifanc yn canolbwyntio ar gyflwyno geiriau newydd. Animation ...')
('latest_episode_url: ', '/programmes/p02b4jth')
('recommendations:', '/programmes/p05mdnfw', 'https://www.bbc.co.uk/programmes/p05mdnfw', 'title: ', None, 'alt title: ', u'  Mabinogi-ogi!  \u2014 Blodeuwedd ', u" Yr wythnos yma, y drwg a'r da, y dylluan a'r eryr, yr hwyl arferol a ch\xe2n yn stori Blod... ")
('recommendations:', '/programmes/p03cnt7m', 'https://www.bbc.co.uk/programmes/p03cnt7m', 'title: ', None, 'alt title: ', u'  Dreigiau Berc  \u2014 Dreigiau: Gwarchodwyr Berc, Triawd y Rhuo ', u' Mae Igion a Stoic yn darganfod tri Tharanfraw ieuanc amddifad ar ynys fechan ac mae Igi... ')
('recommendations:', '/programmes/p06xmw4d', 'https://www.bbc.co.uk/programmes/p06xmw4d', 'title: ', None, 'alt title: ', u'  Ffeil  \u2014 Pennod 205 ', u' Newyddion a chwaraeon dyddiol i bobl ifanc. Daily n

('latest_episode_url: ', '/programmes/b0bxc1sn#credits')
{u'Author': u'Agatha Christie', u'Sir Carmichael Clarke': u'Christopher Villiers', u'Writer': u'Sarah Phelps', u'Rose Marbury': u'Shirley Henderson', u'Inspector Crome': u'Rupert Grint', u'Newsreader': u'Jonathan Keeble', u'Nurse': u'Gillian Waugh', u'Sergeant Yelland': u'Michael Shaeffer', u'Weddle': u'Alexander Kirk', u'German Officer': u'Ali Cook', u'Cust': u'Eamon Farren', u'Prynne': u'Henry Goodman', u'Lily Marbury': u'Anya Chalotra', u'Director': u'Alex Gabassi', u'Yvonne': u'Barbara Drennan', u'Lady Hermione Clarke': u'Tara Fitzgerald', u'Executive Producer': u'Elizabeth Kilgarriff', u'Production Company': u'Agatha Christie Productions Limited', u'Franklin Clarke': u'Andrew Buchan', u'Beautiful Young Man': u'Conrad McCroddan', u'Young Journalist': u'James Cooney', u'Hercule Poirot': u'John Malkovich', u'Betty Barnard': u'Eve Austin', u'Capstick': u'Suzanne Packer', u'Thora Grey': u'Freya Mavor', u'Producer': u'Farah Abushw



('link: ', '/iplayer/episode/b054fm6n/a-cook-abroad-6-rachel-khoos-malaysia')
('title: ', u'A Cook Abroad')
('synopsis: ', u'Celebrity chefs try out different recipes from around the world.')
('latest_episode_url: ', '/programmes/b054fm6n#credits')
{u'Presenter': u'Rachel Khoo', u'Series Producer': u'Katy Fryer', u'Executive Producer': u'Gill Tierney'}
('genre: ', [[u'Factual', '/programmes/genres/factual'], [u'Food & Drink', '/programmes/genres/factual/foodanddrink']])
('format: ', [])
('Left to watch: ', u'11 days left to watch', 'duration: ', u'\n1 hour\n            ')
('long_synopsis: ', [u"Cook and food writer Rachel Khoo heads to Malaysia, home of her extended family, to explore one of the world's most exciting cuisines. From the tropical shores of a remote fishing village to a foraged feast deep in the jungle, Rachel travels across this multicultural country to find out whether good food can unite a nation.", u"On this journey of discovery Rachel delves deep into her own famil

('recommendations:', '/programmes/b04mqc4z', 'https://www.bbc.co.uk/programmes/b04mqc4z', 'title: ', None, 'alt title: ', u'  Spider House  ', u' Alice Roberts enters a spider-filled house to find out about the eight-legged creatures. ')
('recommendations:', '/programmes/b03jrxhv', 'https://www.bbc.co.uk/programmes/b03jrxhv', 'title: ', None, 'alt title: ', u'  Light and Dark  \u2014 Dark ', u' 2/2 Prof Jim Al-Khalili investigates the 99 per cent of the cosmos that is hidden in the dark. ')
('recommendations:', '/programmes/p031d2k6', 'https://www.bbc.co.uk/programmes/p031d2k6', 'title: ', None, 'alt title: ', u"  Life on Air - David Attenborough's 50 Years in Television  ", u' Profiling the television career of David Attenborough, from executive to explorer. ')
('recommendations:', '/programmes/b0078hny', 'https://www.bbc.co.uk/programmes/b0078hny', 'title: ', None, 'alt title: ', u'  Weird Nature  \u2014 Peculiar Potions ', u' 6/6 A look at how a number of creatures take substances f

('latest_episode_url: ', '/programmes/p00y02bb')
/programmes/p00y02bb/episodes
('available episodes: ', u'(9)')
None


('link: ', '/iplayer/episode/b01p015b/the-adventures-of-abney-teal-series-2-19-abneys-precious-things')
('title: ', u'The Adventures of Abney & Teal')
('synopsis: ', u'Animated adventures of two friends who live on an island in the middle of the big city.')
('latest_episode_url: ', '/programmes/b01p015b#credits')


('link: ', '/iplayer/episode/p03lxghc/afoot-again-in-the-past-ickworth-house')
('title: ', u'Afoot Again In The Past')
('synopsis: ', u"Investigating Britain's architectural heritage.")
('latest_episode_url: ', '/programmes/p03lxghc#credits')
{u'Presenter': u'Nigel Havers', u'Series Producer': u'Tim Dunn'}
('genre: ', [[u'Factual', '/programmes/genres/factual'], [u'History', '/programmes/genres/factual/history'], [u'Factual', '/programmes/genres/factual'], [u'Homes & Gardens', '/programmes/genres/factual/homesandgardens'], [u'Homes', '/programmes/genres/fact

('available episodes: ', u'(9)')
('https://www.bbc.co.uk/iplayer/episode/p061252p', u'29 o ddyddiau ar \xf4l i wylio (Gwen 22 Chwefror 2019, 07:25)')
('episode_oneline_synopsis: ', u'\nHeddiw, mae mwy o f\xf4r-ladron o Ysgol L\xf4n Las, Abertawe yn ymuno \xe2 Ben Dant a Cadi i her...\n')
Ysgol Lôn Las, Llansamlet (2)
('https://www.bbc.co.uk/iplayer/episode/p060bfn4', u'24 o ddyddiau ar \xf4l i wylio (Sul 17 Chwefror 2019, 07:30)')
('episode_oneline_synopsis: ', u"\nM\xf4r-ladron o Ysgol Rhyd y Grug, Aberfan sy'n ymuno \xe2 Ben Dant a Cadi i herio Capten Cne...\n")
Ysgol Rhyd Y Grug, Aberfan
('https://www.bbc.co.uk/iplayer/episode/p05zlsp3', u'22 o ddyddiau ar \xf4l i wylio (Gwen 15 Chwefror 2019, 07:25)')
('episode_oneline_synopsis: ', u"\nHeddiw m\xf4r-ladron o Ysgol Sant Baruc, Y Barri sy'n ymuno \xe2 Ben Dant a Cadi i herio Capt...\n")
Ysgol Sant Baruc, Y Barri
('https://www.bbc.co.uk/iplayer/episode/p05yvsfh', u'17 o ddyddiau ar \xf4l i wylio (Sul 10 Chwefror 2019, 07:25)')
('episo

('available episodes: ', u'(2)')
('https://www.bbc.co.uk/iplayer/episode/b04yq4zn', '29 days left to watch (Fri 22 February 2019, 19:00)')
('episode_oneline_synopsis: ', u'\n2/26 Amazing facts about Dotty, a spotted dolphin who lives with close family in the ocean.\n')
('episode_no: ', 'Episode 2 of 26')
Dotty the Spotted Dolphin
('https://www.bbc.co.uk/iplayer/episode/b04xhv00', '27 days left to watch (Wed 20 February 2019, 19:00)')
('episode_oneline_synopsis: ', u'\n1/26 Amazing facts about Rogan, a young joey living in the Australian outback.\n')
('episode_no: ', 'Episode 1 of 26')
Rogan the Red Kangaroo
None


('link: ', '/iplayer/episode/m0001rz0/all-aboard-the-great-reindeer-migration')
('title: ', u'All Aboard! The Great Reindeer Migration')
('synopsis: ', u'Follow the migration of Norway\u2019s Sami reindeer herds across Finnmark.')


KeyboardInterrupt: 